In [1]:
#1. import libraries
import pandas as pd

In [2]:
# 2. Load datasets
games_paid_users = pd.read_csv('games_paid_users.csv')
games_payments = pd.read_csv('games_payments.csv')

In [3]:
# 3. Convert payment_date to datetime
games_payments['payment_date'] = pd.to_datetime(games_payments['payment_date'])

In [4]:
# 4. Add payment_month
games_payments['payment_month'] = games_payments['payment_date'].values.astype('datetime64[M]')

In [5]:
# 5. Aggregate payments per user/game/month
monthly_revenue = games_payments.groupby(['user_id', 'game_name', 'payment_month'], as_index=False)[
    'revenue_amount_usd'].sum()
monthly_revenue.rename(columns={'revenue_amount_usd': 'total_revenue'}, inplace=True)

In [6]:
# 6. Create full monthly calendar per user/game
min_month = monthly_revenue['payment_month'].min()
max_month = monthly_revenue['payment_month'].max()
all_months = pd.date_range(min_month, max_month, freq='MS')

unique_users_games = monthly_revenue[['user_id', 'game_name']].drop_duplicates()
calendar = unique_users_games.assign(key=1).merge(pd.DataFrame({'payment_month': all_months, 'key': 1}), on='key').drop(
    'key', axis=1)

In [7]:
# 7. Merge to fill in missing months with zero revenue
full_data = calendar.merge(monthly_revenue, on=['user_id', 'game_name', 'payment_month'], how='left')
full_data['total_revenue'] = full_data['total_revenue'].fillna(0)

In [8]:
# 8. Sort for further processing
full_data = full_data.sort_values(by=['user_id', 'game_name', 'payment_month'])

In [9]:
# 9. total_revenue_previous
full_data['total_revenue_previous'] = full_data.groupby(['user_id', 'game_name'])['total_revenue'].shift(1).fillna(0)

In [10]:
# 10. total_revenue_previous
def determine_status(row):
    if row['total_revenue'] > 0:
        if row['total_revenue_previous'] == 0:
            # Check first payment
            user_game_payments = full_data[
                (full_data['user_id'] == row['user_id']) &
                (full_data['game_name'] == row['game_name']) &
                (full_data['payment_month'] < row['payment_month'])
                ]
            if user_game_payments['total_revenue'].sum() == 0:
                return 'new'
            return 'back'
        return 'active'
    else:
        if row['total_revenue_previous'] > 0:
            return 'churn'
        return 'deactive'


full_data['status'] = full_data.apply(determine_status, axis=1)

In [11]:
# 11. Merge with user attributes
full_data[['user_id', 'game_name']] = full_data[['user_id', 'game_name']].astype(str)
games_paid_users[['user_id', 'game_name']] = games_paid_users[['user_id', 'game_name']].astype(str)

missing_cols = games_paid_users.columns.difference(full_data.columns)
full_data = full_data.merge(games_paid_users[missing_cols.tolist() + ['user_id', 'game_name']],
                            on=['user_id', 'game_name'], how='left')

In [13]:
# 12. Save to CSV
full_data.to_csv('final_games_dataset.csv', index=False)